In [2]:
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import re
from collections import Counter
import scipy.stats
import seaborn as sns

In [3]:
path11 = r'E:\netData\MAGzip\MAG Nobel\2 Nobelpapers\712 NobelpapersCD.zip'
path12 = r'E:\netData\MAGzip\MAG Nobel\2 Nobelpapers\712 Nobelpapers.zip'
path21 = r'E:\netData\MAGzip\MAG Nobel\2 NobelpaperControls\20954 NobelpapercontrolsCD.zip'
path22 = r'E:\netData\MAGzip\MAG Nobel\2 NobelpaperControls\20954 Nobelpapercontrols.zip'

In [4]:
fNobel1 = pd.read_csv(path11)
fControl1 = pd.read_csv(path21)
fNobel2 = pd.read_csv(path12)
fControl2 = pd.read_csv(path22)

In [5]:
fNobel = pd.concat([fNobel1 , fNobel2] , axis = 1)
fControl = pd.concat([fControl1 , fControl2] , axis = 1)

fNobel = fNobel.loc[:, ~fNobel.columns.duplicated()]
fControl = fControl.loc[:, ~fControl.columns.duplicated()]

In [6]:
fNobel.fillna(0 , inplace=True)
fControl.fillna(0 , inplace=True)

In [7]:
fControl0 = fControl[['Field','cit','CD', 'DIsharp', 'DIeasy','C-CD P', 'ni']]
fControl0['If Prize'] = 0
fControl0.columns = ['Field','C','CD', 'DI*', 'sDI','C-CD P', 'DC','If Prize']

C:\Users\nd\AppData\Local\Temp\ipykernel_19332\1054060326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fControl0['If Prize'] = 0


In [8]:
fNobel0 = fNobel[['Field','cit','CD', 'DIsharp', 'DIeasy','C-CD P', 'ni']]
fNobel0['If Prize'] = 1
fNobel0.columns = ['Field','C','CD', 'DI*', 'sDI','C-CD P', 'DC','If Prize']

C:\Users\nd\AppData\Local\Temp\ipykernel_19332\2285804660.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fNobel0['If Prize'] = 1


In [9]:
fNobel1 = fNobel0[fNobel0['Field']=='Physics']
fNobel2 = fNobel0[fNobel0['Field']=='Chemistry']
fNobel3 = fNobel0[fNobel0['Field']=='Medicine']

In [10]:
fControl1 = fControl0[fControl0['Field']=='Physics']
fControl2 = fControl0[fControl0['Field']=='Chemistry']
fControl3 = fControl0[fControl0['Field']=='Medicine']

In [11]:
ff1 = pd.concat([fNobel1 , fControl1])
ff2 = pd.concat([fNobel2 , fControl2])
ff3 = pd.concat([fNobel3 , fControl3])

In [13]:
ff1.index = range(len(ff1))
ff2.index = range(len(ff2))
ff3.index = range(len(ff3))

# task

In [12]:
def calculate_metrics(df):
    results = []
    
    # Iterate over each column (except the target column)
    for column in df.columns[1:-1]:
        # Sort dataframe by the column in descending order
        sorted_df = df.sort_values(by=column, ascending=False)
        
        precision_values = []
        recall_values = []
        f1_values = []
        
        # Calculate precision, recall, and F1 score for each value of k
        for k in range(1, len(df)+1):
            # Get the top k predictions based on the current column
            predictions = sorted_df['If Prize'].head(k).index
            
            # Calculate true positives, false positives, and false negatives
            true_positives = df.loc[predictions]['If Prize'].sum()
            false_positives = k - true_positives
            false_negatives = df['If Prize'].sum() - true_positives
            
            # Calculate precision, recall, and F1 score
            
            precision = true_positives / (true_positives + false_positives)
            recall = true_positives / (true_positives + false_negatives)
            if precision + recall!=0:
                f1 = 2 * (precision * recall) / (precision + recall)
            else:
                f1 = 0
            
            precision_values.append(precision)
            recall_values.append(recall)
            f1_values.append(f1)
        
        # Calculate the average precision, recall, and F1 score for all k values
        avg_precision = np.mean(precision_values)
        avg_recall = np.mean(recall_values)
        avg_f1 = np.mean(f1_values)
        
        # Append results to the list
        results.append([column, avg_precision, avg_recall, avg_f1])
    
    # Create a new dataframe with the results
    result_df = pd.DataFrame(results, columns=['Column', 'Average Precision', 'Average Recall', 'Average F1'])
    
    return result_df

In [14]:
result_df1 = calculate_metrics(ff1)

In [20]:
result_df2 = calculate_metrics(ff2)

In [21]:
result_df3 = calculate_metrics(ff3)

In [17]:
result_df1

,Column,Average Precision,Average Recall,Average F1
0,C,0.130808,0.942023,0.187956
1,CD,0.024231,0.467482,0.045870
2,DI*,0.024946,0.486705,0.047325
3,sDI,0.022225,0.444217,0.042184
4,C-CD P,0.104411,0.749182,0.144983
5,DC,0.133648,0.945655,0.190670


In [22]:
result_df2

,Column,Average Precision,Average Recall,Average F1
0,C,0.086202,0.908523,0.128635
1,CD,0.016284,0.475859,0.031431
2,DI*,0.017142,0.505006,0.033101
3,sDI,0.014328,0.444313,0.027726
4,C-CD P,0.052903,0.662897,0.077250
5,DC,0.089017,0.919042,0.132037


In [23]:
result_df3

,Column,Average Precision,Average Recall,Average F1
0,C,0.145050,0.928360,0.206962
1,CD,0.039005,0.558683,0.072309
2,DI*,0.041146,0.596275,0.076400
3,sDI,0.036665,0.552280,0.068314
4,C-CD P,0.084791,0.672383,0.120105
5,DC,0.149697,0.935929,0.211786
